<a href="https://colab.research.google.com/github/ReallyJaegar/CPbuddy/blob/main/char_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

scrapping data

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import os

In [ ]:
handle = input("Type Your Handle ")

Type Your Handle tourist


In [ ]:
for i in range (1,3):
    # print('grabbing data')
    gen_url='https://codeforces.com/submissions/'+handle+'/page/'+str(i)
#     print(gen_url)
    r=urlopen(gen_url)
#     print('done')

In [ ]:
problem_id=[]
no_name=[]
soup=BeautifulSoup(r)
co=soup.find_all('td')

for row in range(1,len(co)):
    if len(co[row].attrs)==4:
        if len(co[row].span.attrs)==5:
            if co[row].span.attrs['submissionverdict'] == 'OK':
                problem_id.append(co[row].span.attrs['submissionid'])
                no_name.append(row)


contest_id = []
for j in range(0,len(no_name)):
    query = no_name[j]
    contest_id.append(co[query-2].a.attrs['href'].split('/')[2])                

In [ ]:
for j in range(0, len(contest_id)):
    url = 'http://codeforces.com/contest/'+contest_id[j]+'/submission/'+problem_id[j]
    r =urlopen(url)
    soup = BeautifulSoup(r)
    var = soup.find_all('td')
    problem_name = var[2].a['href'].split('/')[4]
#     print(var[2].a['href'].split('/')[4])
#     print('getting code '+contest_id[j]+'-'+problem_name)
    
    co=soup.find_all('div')
    target=open('collect.txt','a')
    
    for row in co[3].find_all('pre',attrs={"class" : "program-source"}):
        target.write(row.text+'\n\n')
        
#     print('got code'+contest_id[j]+'-'+problem_name)
    target.close()

In [ ]:
f=open('collect.txt')
Collection=f.read()

text-encoding using character level encoding instead of word (i was stuck here for so many days, as i was using modules which allows word level NLP)

In [ ]:
text_vec_layer=tf.keras.layers.TextVectorization(split="character",standardize="lower")
text_vec_layer.adapt([Collection])
encoded=text_vec_layer([Collection])[0]

NameError: ignored

In [ ]:
encoded-=2
n_tokens=text_vec_layer.vocabulary_size()-2
dataset_size=len(encoded)

In [ ]:
print(n_tokens)

67


function to convert a long sequence of character IDs into a dataset of input/target window pairs

In [ ]:
def to_dataset(sequence, length,shuffle=False, seed=None, batch_size=32):
  ds=tf.data.Dataset.from_tensor_slices(sequence)
  ds=ds.window(length+1,shift=1,drop_remainder=True)
  ds=ds.flat_map(lambda window_ds:window_ds.batch(length+1))
  if shuffle:
    ds=ds.shuffle(buffer_size=100_000,seed=seed)

  ds=ds.batch(batch_size)
  return ds.map(lambda window: (window[:,:-1],window[:,1:])).prefetch(1)

training set and validating set

In [ ]:
length=100
tf.random.set_seed(42)
train_set=to_dataset(encoded[:1_000_000],length=length,shuffle=True,seed=42)
valid_set=to_dataset(encoded[1_000_000:1_060_000],length=length)
test_set=to_dataset(encoded[1_060_000:],length=length)

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089
